# load dataset file

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server
# https://drive.google.com/file/d/10yi1MvU3lrlqyGLGZ0Ee7o4e72zEW4Fg/view?usp=sharing
#chats.csv DOTA dataset 1.4 million
# https://drive.google.com/file/d/1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj/view?usp=sharing
# https://drive.google.com/file/d/1-HsJ2w944FNCZPPiqYjV7Ip-2fmdNqz5/view?usp=sharing
download_files = [
          {'id':'1gNfBqguzBu8cHKMPc8C44GbvD443dNC5','file_name':'Wassem.csv'},
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("Wassem.csv")

In [ ]:
data.head()

,Unnamed: 0,Tweet_ID,Previous,User_ID,Text,Label
0,0,567080871472017408,5.670786e+17,2941145694,@truaemusic The followers of the religion give...,racism
1,1,566098518616449024,5.660874e+17,2941145694,@Love___Egypt @washingtonpost All the land tha...,racism
2,2,554672350361571328,NaN,2941145694,@CamilleHughes16 @edbarber101658 @_Watch_Obser...,racism
3,3,569878129772081153,5.698451e+17,2941145694,@KellyTurner99 @brushyblues @buellerishere The...,racism
4,4,568541747849998337,NaN,2941145694,@buellerishere Who cares. ISIS does what Moham...,racism


In [ ]:
map_dict = {'racism':1,'sexism':1,'none':0}
def map_to_binary(x):
  return map_dict[x]
data['Label'] = data['Label'].map(map_to_binary)
data.head()

,Unnamed: 0,Tweet_ID,Previous,User_ID,Text,Label
0,0,567080871472017408,5.670786e+17,2941145694,@truaemusic The followers of the religion give...,1
1,1,566098518616449024,5.660874e+17,2941145694,@Love___Egypt @washingtonpost All the land tha...,1
2,2,554672350361571328,NaN,2941145694,@CamilleHughes16 @edbarber101658 @_Watch_Obser...,1
3,3,569878129772081153,5.698451e+17,2941145694,@KellyTurner99 @brushyblues @buellerishere The...,1
4,4,568541747849998337,NaN,2941145694,@buellerishere Who cares. ISIS does what Moham...,1


In [ ]:
data['Label'].value_counts()

0    11115
1     5087
Name: Label, dtype: int64

# bert tokenizer

In [ ]:
!pip install transformers

In [ ]:
import random
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda')

ModuleNotFoundError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X = data['Text'].tolist()
y = data['Label'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=28)
len(X_train)

14581

In [ ]:
len(X_test)

1621

In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# Function to get token ids for a list of texts 
def encode_fn(text_list):
    all_input_ids = []    
    for text in text_list:
        input_ids = tokenizer.encode(
                        text,                      
                        add_special_tokens = True,  # 添加special tokens， 也就是CLS和SEP
                        max_length = 50,           # 设定最大文本长度
                        pad_to_max_length = True,   # pad到最大的长度  
                        return_tensors = 'pt'       # 返回的类型为pytorch tensor
                   )
        all_input_ids.append(input_ids)    
    all_input_ids = torch.cat(all_input_ids, dim=0)
    return all_input_ids

# torch dataset

In [ ]:
epochs = 5
batch_size = 32

# Split data into train and validation
train_ids = encode_fn(X_train)
train_labels = torch.tensor(y_train)
train_dataset = TensorDataset(train_ids, train_labels)

test_ids = encode_fn(X_test)
test_labels = torch.tensor(y_test)
testdataset = TensorDataset(test_ids, test_labels)

# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create train and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testdataset, batch_size = batch_size, shuffle = False)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# construct model

In [ ]:
# Load the pretrained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False)
model.cuda()

# create optimizer and learning rate schedule
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    
    """A function for calculating accuracy scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return labels_flat, pred_flat

# model training

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss, total_val_loss = 0, 0
    total_eval_accuracy = 0
    for step, batch in enumerate(train_dataloader):
        model.zero_grad()
        # loss, logits = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
        outputs = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step() 
        scheduler.step()
        
    model.eval()
    labels_all = []
    pred_flat_all = []
    for i, batch in enumerate(test_dataloader):
        with torch.no_grad():
            outputs = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
            loss = outputs.loss
            logits = outputs.logits
                
            total_val_loss += loss.item()
            
            logits = logits.detach().cpu().numpy()
            label_ids = batch[1].to('cpu').numpy()
            labels_flat, pred_flat = flat_accuracy(logits, label_ids)
            labels_all.extend(labels_flat)
            pred_flat_all.extend(pred_flat)

    torch.save(model, 'model_bert_base_from_wassem'+str(epoch)+'.ckpt')
    # torch.save(slot_model, 'model_slot'+str(epoch)+'.ckpt')
    
    avg_train_loss = total_loss / len(train_dataloader)
    avg_val_loss = total_val_loss / len(test_dataloader)
    avg_val_accuracy = accuracy_score(labels_all, pred_flat_all)
    
    print(f'Train loss     : {avg_train_loss}')
    print(f'Validation loss: {avg_val_loss}')
    print(f'Accuracy: {avg_val_accuracy:.4f}')
    print('\n')

Train loss     : 0.3630503141566327
Validation loss: 0.32349565917370365
Accuracy: 0.8600


Train loss     : 0.24576621659492193
Validation loss: 0.32729288263648165
Accuracy: 0.8698


Train loss     : 0.15850132502198808
Validation loss: 0.425275747682534
Accuracy: 0.8482


Train loss     : 0.10168853756081182
Validation loss: 0.4747424707839302
Accuracy: 0.8637


Train loss     : 0.06722365229762274
Validation loss: 0.4971212418932541
Accuracy: 0.8637




# evaluate on 45k dota dataset

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server
# https://drive.google.com/file/d/10yi1MvU3lrlqyGLGZ0Ee7o4e72zEW4Fg/view?usp=sharing
#chats.csv DOTA dataset 1.4 million
# https://drive.google.com/file/d/1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj/view?usp=sharing
# https://drive.google.com/file/d/1TJVC6VGiAqrMzNbh_-a70FWlR6zzLuvP/view?usp=sharing
download_files = [
          {'id':'1TJVC6VGiAqrMzNbh_-a70FWlR6zzLuvP','file_name':'45k_add_slot_label.csv'},
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("45k_add_slot_label.csv")
data.head()

,Unnamed: 0,match_id,conversation_id,utterance,time,slot,unit,Label,combined_annotation,final_label,review_label,word_level_data,slot_label,combined_annotation_version2
0,0,0,0,force it,-8,6,6k Slayer,A,"force (D), it (P),",A,NaN,force it,D P,"force (D), it (P),"
1,1,0,0,space created [SEPA] hah,5,1,Monkey,O,"space (O), created (O), [SEPA] (SEPA), hah (O),",O,NaN,space created [SEPA] hah,O O SEPA O,"space (O), created (O), [SEPA] (SEPA), hah (O),"
2,2,0,0,ez 500,9,6,6k Slayer,I,"ez (S), 500 (O),",I,NaN,ez 500,S O,"ez (S), 500 (O),"
3,3,0,1,mvp ulti,934,4,Kira,O,"mvp (O), ulti (S),",O,NaN,mvp ulti,O S,"mvp (O), ulti (S),"
4,4,0,2,bye,1486,6,6k Slayer,O,"bye (O),",O,NaN,bye,O,"bye (O),"


In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# colnames = ['1','2','3','utterance','4','5','6','Label','combined_annotation']
# test_data = pd.read_csv("50K_Output.csv",names=colnames, header=None)

In [ ]:
test_data = data
def map_to_binary(x):
  if x=='E' or x =='I':
    return 1
  else:
    return 0

test_data['Label'] = test_data['final_label'].map(map_to_binary)
test_data.head()

,Unnamed: 0,match_id,conversation_id,utterance,time,slot,unit,Label,combined_annotation,final_label,review_label,word_level_data,slot_label,combined_annotation_version2
0,0,0,0,force it,-8,6,6k Slayer,0,"force (D), it (P),",A,NaN,force it,D P,"force (D), it (P),"
1,1,0,0,space created [SEPA] hah,5,1,Monkey,0,"space (O), created (O), [SEPA] (SEPA), hah (O),",O,NaN,space created [SEPA] hah,O O SEPA O,"space (O), created (O), [SEPA] (SEPA), hah (O),"
2,2,0,0,ez 500,9,6,6k Slayer,1,"ez (S), 500 (O),",I,NaN,ez 500,S O,"ez (S), 500 (O),"
3,3,0,1,mvp ulti,934,4,Kira,0,"mvp (O), ulti (S),",O,NaN,mvp ulti,O S,"mvp (O), ulti (S),"
4,4,0,2,bye,1486,6,6k Slayer,0,"bye (O),",O,NaN,bye,O,"bye (O),"


In [ ]:
def str_context(x):
  return str(x)

test_data['utterance'] = test_data['utterance'].map(str_context)
test_data.head()

,Unnamed: 0,match_id,conversation_id,utterance,time,slot,unit,Label,combined_annotation,final_label,review_label,word_level_data,slot_label,combined_annotation_version2
0,0,0,0,force it,-8,6,6k Slayer,0,"force (D), it (P),",A,NaN,force it,D P,"force (D), it (P),"
1,1,0,0,space created [SEPA] hah,5,1,Monkey,0,"space (O), created (O), [SEPA] (SEPA), hah (O),",O,NaN,space created [SEPA] hah,O O SEPA O,"space (O), created (O), [SEPA] (SEPA), hah (O),"
2,2,0,0,ez 500,9,6,6k Slayer,1,"ez (S), 500 (O),",I,NaN,ez 500,S O,"ez (S), 500 (O),"
3,3,0,1,mvp ulti,934,4,Kira,0,"mvp (O), ulti (S),",O,NaN,mvp ulti,O S,"mvp (O), ulti (S),"
4,4,0,2,bye,1486,6,6k Slayer,0,"bye (O),",O,NaN,bye,O,"bye (O),"


In [ ]:
test_ids = encode_fn(test_data['utterance'].tolist())
test_labels = torch.tensor(test_data['Label'].tolist())
testdataset = TensorDataset(test_ids, test_labels)

# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create train and validation dataloaders
# train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testdataset, batch_size = batch_size, shuffle = False)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
labels_all = []
pred_flat_all = []
for i, batch in enumerate(test_dataloader):
    with torch.no_grad():
        outputs = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
        loss = outputs.loss
        logits = outputs.logits
            
        total_val_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = batch[1].to('cpu').numpy()
        labels_flat, pred_flat = flat_accuracy(logits, label_ids)
        labels_all.extend(labels_flat)
        pred_flat_all.extend(pred_flat)
print(len(labels_all))
print(len(pred_flat_all))

44869
44869


In [ ]:
avg_val_accuracy = accuracy_score(labels_all, pred_flat_all)
avg_val_accuracy

0.8059907731395841

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=labels_all,y_pred=pred_flat_all,digits=4))

              precision    recall  f1-score   support

           0     0.8062    0.9985    0.8921     36040
           1     0.7650    0.0203    0.0395      8829

    accuracy                         0.8060     44869
   macro avg     0.7856    0.5094    0.4658     44869
weighted avg     0.7981    0.8060    0.7243     44869



# evaluate on dota dataset

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server
# https://drive.google.com/file/d/10yi1MvU3lrlqyGLGZ0Ee7o4e72zEW4Fg/view?usp=sharing
#chats.csv DOTA dataset 1.4 million
# https://drive.google.com/file/d/1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj/view?usp=sharing
# https://drive.google.com/file/d/1-HsJ2w944FNCZPPiqYjV7Ip-2fmdNqz5/view?usp=sharing
download_files = [
          {'id':'1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj','file_name':'50K_Output.csv'},
          {'id':'1-HsJ2w944FNCZPPiqYjV7Ip-2fmdNqz5','file_name':'word_level_data_combined_annotation.csv'},
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
colnames = ['1','2','3','utterance','4','5','6','Label','combined_annotation']
test_data = pd.read_csv("50K_Output.csv",names=colnames, header=None)

In [ ]:
def map_to_binary(x):
  if x=='E' or x =='I':
    return 1
  else:
    return 0

test_data['Label'] = test_data['Label'].map(map_to_binary)
test_data.head()

,1,2,3,utterance,4,5,6,Label,combined_annotation
0,0,0,0,force it,-8,6,6k Slayer,0,"force (D), it (P),"
1,1,0,0,space created [SEPA] hah,5,1,Monkey,0,"space (O), created (O), [SEPA] (SEPA), hah (O),"
2,2,0,0,ez 500,9,6,6k Slayer,1,"ez (S), 500 (O),"
3,3,0,1,mvp ulti,934,4,Kira,0,"mvp (O), ulti (S),"
4,4,0,2,bye,1486,6,6k Slayer,0,"bye (O),"


In [ ]:
def str_context(x):
  return str(x)

test_data['utterance'] = test_data['utterance'].map(str_context)
test_data.head()

,1,2,3,utterance,4,5,6,Label,combined_annotation
0,0,0,0,force it,-8,6,6k Slayer,0,"force (D), it (P),"
1,1,0,0,space created [SEPA] hah,5,1,Monkey,0,"space (O), created (O), [SEPA] (SEPA), hah (O),"
2,2,0,0,ez 500,9,6,6k Slayer,1,"ez (S), 500 (O),"
3,3,0,1,mvp ulti,934,4,Kira,0,"mvp (O), ulti (S),"
4,4,0,2,bye,1486,6,6k Slayer,0,"bye (O),"


In [ ]:
test_ids = encode_fn(test_data['utterance'].tolist())
test_labels = torch.tensor(test_data['Label'].tolist())
testdataset = TensorDataset(test_ids, test_labels)

# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create train and validation dataloaders
# train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testdataset, batch_size = batch_size, shuffle = False)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
labels_all = []
pred_flat_all = []
for i, batch in enumerate(test_dataloader):
    with torch.no_grad():
        loss, logits = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
            
        total_val_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = batch[1].to('cpu').numpy()
        labels_flat, pred_flat = flat_accuracy(logits, label_ids)
        labels_all.extend(labels_flat)
        pred_flat_all.extend(pred_flat)
print(len(labels_all))
print(len(pred_flat_all))

50001
50001


In [ ]:
avg_val_accuracy = accuracy_score(labels_all, pred_flat_all)
avg_val_accuracy

0.8307233855322893

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=labels_all,y_pred=pred_flat_all,digits=4))

              precision    recall  f1-score   support

           0     0.8314    0.9987    0.9074     41508
           1     0.6043    0.0099    0.0195      8493

    accuracy                         0.8307     50001
   macro avg     0.7178    0.5043    0.4634     50001
weighted avg     0.7928    0.8307    0.7565     50001



# evaluate on storm dataset

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server
# https://drive.google.com/file/d/10yi1MvU3lrlqyGLGZ0Ee7o4e72zEW4Fg/view?usp=sharing
#chats.csv DOTA dataset 1.4 million
# https://drive.google.com/file/d/1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj/view?usp=sharing
# https://drive.google.com/file/d/1-HsJ2w944FNCZPPiqYjV7Ip-2fmdNqz5/view?usp=sharing
download_files = [
          {'id':'1LNjeEeHNpJwS4O9OK2gOSpwddwzHCaqW','file_name':'storm.csv'},
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# colnames = ['1','2','3','utterance','4','5','6','Label','combined_annotation']
test_data = pd.read_csv("storm.csv")

In [ ]:
map_dict = {'hate':1,'noHate':0,'relation':0,'idk/skip':0}
def map_to_binary(x):
  return map_dict[x]
test_data['label'] = test_data['label'].map(map_to_binary)
test_data.head()

,sentence,label,user_id,file_id
0,"b'As of March 13th , 2014 , the booklet had be...",0,572066,12834217_1
1,b'In order to help increase the booklets downl...,0,572066,12834217_2
2,b'( Simply copy and paste the following text i...,0,572066,12834217_3
3,b'Click below for a FREE download of a colorfu...,1,572066,12834217_4
4,"b""Click on the `` DOWNLOAD ( 7.42 MB ) '' gree...",0,572066,12834217_5


In [ ]:
test_ids = encode_fn(test_data['sentence'].tolist())
test_labels = torch.tensor(test_data['label'].tolist())
testdataset = TensorDataset(test_ids, test_labels)

# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create train and validation dataloaders
# train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testdataset, batch_size = batch_size, shuffle = False)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
labels_all = []
pred_flat_all = []
for i, batch in enumerate(test_dataloader):
    with torch.no_grad():
        loss, logits = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
            
        total_val_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = batch[1].to('cpu').numpy()
        labels_flat, pred_flat = flat_accuracy(logits, label_ids)
        labels_all.extend(labels_flat)
        pred_flat_all.extend(pred_flat)
print(len(labels_all))
print(len(pred_flat_all))

10944
10944


In [ ]:
avg_val_accuracy = accuracy_score(labels_all, pred_flat_all)
avg_val_accuracy

0.8845029239766082

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=labels_all,y_pred=pred_flat_all,digits=4))

              precision    recall  f1-score   support

           0     0.8985    0.9811    0.9380      9748
           1     0.3867    0.0970    0.1551      1196

    accuracy                         0.8845     10944
   macro avg     0.6426    0.5391    0.5465     10944
weighted avg     0.8426    0.8845    0.8525     10944



# evaluate on fox dataset

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server
# https://drive.google.com/file/d/10yi1MvU3lrlqyGLGZ0Ee7o4e72zEW4Fg/view?usp=sharing
#chats.csv DOTA dataset 1.4 million
# https://drive.google.com/file/d/1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj/view?usp=sharing
# https://drive.google.com/file/d/1-HsJ2w944FNCZPPiqYjV7Ip-2fmdNqz5/view?usp=sharing
download_files = [
          {'id':'1fKnmVJYBVkLKSr6YDXwUz5tZAHDc7YvO','file_name':'fox.csv'},
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
test_data = pd.read_csv("fox.csv")

In [ ]:
map_dict = {'hateful':1,'non-hateful':0}
def map_to_binary(x):
  return map_dict[x]
test_data['label'] = test_data['label'].map(map_to_binary)
test_data.head()

,Unnamed: 0,title,text,label,succ,meta,user,mentions,prev
0,0,German lawmakers approve 'no means no' rape la...,Merkel would never say NO,1,NaN,German lawmakers passed a bill Thursday that w...,barryswallows,NaN,NaN
1,1,German lawmakers approve 'no means no' rape la...,"Expect more and more women to be asking .. ""wh...",1,NaN,German lawmakers passed a bill Thursday that w...,PostApocalypticHero,NaN,NaN
2,2,German lawmakers approve 'no means no' rape la...,Groping people in public wasn't already illega...,0,NaN,German lawmakers passed a bill Thursday that w...,californiamojo,NaN,NaN
3,3,German lawmakers approve 'no means no' rape la...,"Merkel, possible the only person in charge who...",1,NaN,German lawmakers passed a bill Thursday that w...,MikeSte,NaN,NaN
4,4,German lawmakers approve 'no means no' rape la...,"They know very well, no means NO! They need to...",1,NaN,German lawmakers passed a bill Thursday that w...,scientist,NaN,NaN


In [ ]:
test_ids = encode_fn(test_data['text'].tolist())
test_labels = torch.tensor(test_data['label'].tolist())
testdataset = TensorDataset(test_ids, test_labels)

# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create train and validation dataloaders
# train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testdataset, batch_size = batch_size, shuffle = False)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
labels_all = []
pred_flat_all = []
for i, batch in enumerate(test_dataloader):
    with torch.no_grad():
        loss, logits = model(batch[0].to(device), token_type_ids=None, attention_mask=(batch[0]>0).to(device), labels=batch[1].to(device))
            
        total_val_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = batch[1].to('cpu').numpy()
        labels_flat, pred_flat = flat_accuracy(logits, label_ids)
        labels_all.extend(labels_flat)
        pred_flat_all.extend(pred_flat)
print(len(labels_all))
print(len(pred_flat_all))

1526
1526


In [ ]:
avg_val_accuracy = accuracy_score(labels_all, pred_flat_all)
avg_val_accuracy

0.72870249017038

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=labels_all,y_pred=pred_flat_all,digits=4))

              precision    recall  f1-score   support

           0     0.7339    0.9734    0.8369      1091
           1     0.6329    0.1149    0.1946       435

    accuracy                         0.7287      1526
   macro avg     0.6834    0.5442    0.5157      1526
weighted avg     0.7051    0.7287    0.6538      1526

